In [ ]:
# Load packages
library(ggplot2)
library(MASS)
options(repr.plot.width=3, repr.plot.height=3)

In [ ]:
# Read the cleaned VIS snapshot data from file
vis.data = read.table(file="vis_snapshots_cleaned.csv", sep=",", header=TRUE)

In [ ]:
# Read biomass data from manually measured plants
manual.st.data = read.table(file='./data/manual_biomass_samples.csv', sep=",", header=TRUE, stringsAsFactors=FALSE)

In [ ]:
# Extract the image data for each sampled plant from the cleaned VIS data set
st.data = merge(manual.st.data, vis.data, by = c('plant_id', 'datetime'))

In [ ]:
# Create an out-of-frame indicator variable
st.data$outind = NA
st.data[st.data$outlier == 'True',]$outind = 1
st.data[st.data$outlier == 'False',]$outind = 0

In [ ]:
# Create a genotype indicator variable
st.data$group = NA
st.data[st.data$genotype == 'A10',]$group = 0
st.data[st.data$genotype == 'B100',]$group = 1

In [ ]:
# Full linear model for fresh-weight biomass. Includes side-view area, top-view area and height
fw.full = lm(fresh_weight ~ sv_area * tv_area * height_above_bound, st.data)
summary(fw.full)

In [ ]:
# Automated step-wise model selection with AIC
fw.step = stepAIC(fw.full, direction="both")
summary(fw.step)

In [ ]:
# Build AIC model
fw.aic = lm(fresh_weight ~ sv_area + height_above_bound, st.data)
summary(fw.aic)

In [ ]:
# The AIC model contains heigh_above_bound which does not have a significant coefficient, test dropping
fw.red = lm(fresh_weight ~ sv_area, st.data)
summary(fw.red)

In [ ]:
# Goodness of fit test
anova(fw.aic, fw.red)

In [ ]:
# Side-view area model
sv.model = lm(fresh_weight ~ sv_area, st.data)
summary(sv.model)

In [ ]:
# Plot fresh-weight biomass linear model
ggplot(st.data,aes(x=sv_area/1e5, y=fresh_weight)) + 
    geom_smooth(method="lm", color="black", formula = y ~ x) +
    geom_point(size=2.5) +
    scale_x_continuous("Shoot and leaf area (x10^5 px)") +
    scale_y_continuous("Fresh-weight biomass (g)") +
    theme_bw() +
    theme(axis.title.x=element_text(face="bold"),
          axis.title.y=element_text(face="bold"))

In [ ]:
# Can we improve the biomass model by accounting for plants grown out of frame?
sv.ind.model = lm(fresh_weight ~ sv_area + outind, st.data)
summary(sv.ind.model)
anova(sv.model, sv.ind.model)

In [ ]:
# Plot fresh-weight biomass linear model with out-of-frame
ggplot(st.data, aes(x=sv_area/1e5, y=fresh_weight, group=outlier, color=outlier)) +
    geom_point(size=2.5) +
    geom_smooth(method="lm", color="black") +
    scale_x_continuous("Shoot and leaf area (x10^5 px)") +
    scale_y_continuous("Fresh-weight biomass (g)") +
    theme_bw() +
    theme(legend.position=c(0.2,0.8),
          axis.title.x=element_text(face="bold"),
          axis.title.y=element_text(face="bold")) +
          labs(color="Out-of-frame")

In [ ]:
# Does our ability to predict biomass depend on genotype?
sv.gt.model = lm(fresh_weight ~ sv_area + group, st.data)
summary(sv.gt.model)

In [ ]:
# Plot the fresh-weight linear model with genotype
ggplot(st.data, aes(x=sv_area/1e5, y=fresh_weight, group=genotype, color=genotype)) +
    geom_point(size=2.5) +
    geom_smooth(method="lm", color="black") +
    scale_x_continuous("Shoot and leaf area (x10^5 px)") +
    scale_y_continuous("Fresh-weight biomass (g)") +
    theme_bw() +
    theme(legend.position=c(0.2,0.8),
          axis.title.x=element_text(face="bold"),
          axis.title.y=element_text(face="bold")) +
          labs(color="Genotype")

In [ ]:
# Dry-weight biomass
dry.sv.model = lm(dry_weight ~ sv_area, st.data)
summary(dry.sv.model)

In [ ]:
# Plot dry-weight biomass model
ggplot(st.data,aes(x=sv_area/1e5, y=dry_weight)) +
    geom_smooth(method="lm", color="black", formula = y ~ x) +
    geom_point(size=2.5) +
    scale_x_continuous("Shoot and leaf area (x10^5 px)") +
    scale_y_continuous("Dry-weight biomass (g)") +
    theme_bw() +
    theme(axis.title.x=element_text(face="bold"),
          axis.title.y=element_text(face="bold"))

In [ ]:
# Use the biomass models to predict biomass for plants not manually measured
# Remove plants that were sampled for biomass measurements
vis.data = vis.data[!vis.data$plant_id %in% st.data$plant_id,]

In [ ]:
# Predict fresh- and dry-weight biomass from linear models
vis.data$fw_biomass = predict.lm(object = sv.model, newdata=vis.data)
vis.data$dw_biomass = predict.lm(object = dry.sv.model, newdata=vis.data)

In [ ]:
# Write the updated table to a file
write.table(vis.data, file = "vis_snapshots_with_biomass.csv", quote = FALSE, sep = ",")